In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

In [2]:
#creates analysis/exploration.json

exploration = rs.explore_data(
    data=data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [3]:
# fits model

model = rs.glm(
    formula="ClaimCount ~ VehPower + VehAge + C(Area) + C(Region)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [4]:
#writes analysis/diagnostics
model.diagnostics(
    data=data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ VehPower + VehAge + C(Area) + C(Region)', 'family': 'NegativeBinomial(theta=1.3802)', 'link': 'log', 'n_observations': 678012, 'n_parameters': 28, 'degrees_of_freedom_residual': 677984, 'converged': True, 'iterations': 6}, convergence_details=ConvergenceDetails(max_iterations_allowed=25, iterations_used=6, converged=True, reason='converged'), fit_statistics={'deviance': 196741.39765171794, 'null_deviance': 192800.37653563166, 'deviance_explained': -0.02044094097169946, 'log_likelihood': -98370.69882585897, 'aic': 196797.39765171794, 'bic': 197117.35141916262, 'pearson_chi2': 1678923.2261339806, 'dispersion_deviance': 0.29018590062850735, 'dispersion_pearson': 2.47634638300311}, loss_metrics={'family_deviance_loss': 0.2901740260018973, 'mse': 0.05512677026308909, 'mae': 0.09871081271150473, 'rmse': 0.2347909075392169}, calibration={'ae_ratio': 0.991, 'hl_pvalue': 0.0, 'problem_deciles': [{'decile': 0, 'ae': 6.44, 'n': 67124, 'ae_c

In [5]:
model = rs.glm(
    formula="ClaimCount ~ C(Area) + bs(VehAge, df=4)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [6]:

model = rs.glm(
    formula="ClaimCount ~ C(Area):bs(VehAge, df=4)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [7]:
print(model.summary())

                                 GLM Results                                  

Family:              NegativeBinomial(theta=1.2840) No. Observations:        678012
Link Function:       (default)       Df Residuals:            677996
Method:              IRLS            Df Model:                    15
Scale:               0.2904          Iterations:                   7

Log-Likelihood:         -268298.8899 Deviance:                196866.1067
AIC:                     536629.7799 Null Deviance:           191531.0335
BIC:                     536812.6106 Pearson chi2:             1747655.92
Converged:           True           

------------------------------------------------------------------------------
Variable                        Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept                    -2.1776     0.0050 -433.279  <0.0001   [ -2.1875,  -2.1678]  ***
Area[T.B]:bs(VehAge, 2

In [2]:
import time

t0 = time.time()
model1 = rs.glm("ClaimCount ~ C(Area) + bs(VehAge, df=4)", data, family="negbinomial", offset="Exposure").fit()
print(f"Additive: {time.time()-t0:.2f}s, Iterations: {model1.iterations}, Converged: {model1.converged}")

t0 = time.time()
model2 = rs.glm("ClaimCount ~ C(Area):bs(VehAge, df=4)", data, family="negbinomial", offset="Exposure").fit()
print(f"Interaction: {time.time()-t0:.2f}s, Iterations: {model2.iterations}, Converged: {model2.converged}")

Additive: 24.29s, Iterations: 25, Converged: False
Interaction: 6.25s, Iterations: 1, Converged: True


In [6]:
# More iterations
model1 = rs.glm("ClaimCount ~ C(Area) + bs(VehAge, df=4)", data, family="negbinomial", offset="Exposure").fit(max_iter=100)
print(f"Iterations: {model1.iterations}, Converged: {model1.converged}")

Iterations: 2, Converged: True


In [2]:
import time

t0 = time.time()
model1 = rs.glm("ClaimCount ~ C(Area) + bs(VehAge, df=4)", data, family="negbinomial", offset="Exposure").fit()
print(f"Additive: {time.time()-t0:.2f}s, Iterations: {model1.iterations}, Converged: {model1.converged}")

t0 = time.time()
model2 = rs.glm("ClaimCount ~ C(Area):bs(VehAge, df=4)", data, family="negbinomial", offset="Exposure").fit()
print(f"Interaction: {time.time()-t0:.2f}s, Iterations: {model2.iterations}, Converged: {model2.converged}")

Additive: 6.12s, Iterations: 1, Converged: True
Interaction: 6.46s, Iterations: 2, Converged: True
